In [1]:
import os
import operator
import math
import re
import time
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pickle
import string

In [2]:
query_file_path = "E:\\CS 6200\\AP_DATA\\query_desc.51-100.short.txt"
queries = dict()
with open(query_file_path) as f:
    for line in f:
        if len(line.strip()) > 0:
            query_no = re.search(r'\d+', line).group()
            line = line.lstrip('0123456789.- ')
            if query_no not in queries:
                queries[query_no] = line.rstrip().replace('-', ' ').replace('.', '').replace('(', '').replace(')', '').replace('"', '').replace(',', '')
            # print(queries)

In [3]:
def unpickle_file(file_name):
    path = 'E:\\CS 6200\\Assignment 2\\stemmed'
    with open(os.path.join(path, file_name), 'rb') as handle:
        b = pickle.load(handle)
    return b

In [4]:
catalog = unpickle_file('l85_catalog.pickle')

In [5]:
doc_lengths = unpickle_file('doc_lengths.pickle')

In [6]:
def avg_doc_length(document_lengths):
    sum = 0
    num_of_docs = len(list(document_lengths.keys()))
    for doc in doc_lengths:
        sum += document_lengths[doc]
    return (sum/num_of_docs)

In [7]:
avg_length = avg_doc_length(doc_lengths)

In [8]:
docid_to_num = dict()
num_to_docid = dict()

In [9]:
def get_docid_mapping(id_mapping_file):
    path = 'E:\\CS 6200\\AP_DATA'
    with open(os.path.join(path, id_mapping_file), 'r') as f:
        for line in f:
            doc_mapping = line.split()
            docid_to_num[doc_mapping[0]] = doc_mapping[1]
            num_to_docid[doc_mapping[1]] = doc_mapping[0]

In [10]:
get_docid_mapping('docmapping.txt')

In [11]:
def get_doc_length(docnum, doc_lengths):
    return doc_lengths[docnum]

In [12]:
total_docs = len(list(doc_lengths.keys()))

In [13]:
print(total_docs)

84678


In [14]:
stopWords = []

In [15]:
with open("E:\\CS 6200\\AP_DATA\\stoplist.txt") as f:
    for line in f:
        line = line.rstrip()
        stopWords.append(line.rstrip())

In [78]:
def get_ttf(word):
    inv_file = 'E:\\CS 6200\\Assignment 2\\stemmed\\l85_merged_index.txt'

    with open(inv_file, 'r') as f:
        f.seek(catalog[word][0])
        line = f.read(catalog[word][1]) 
        line = re.sub('^.*:', '', line)
        line = line.replace(';','')
        numbers_list = line.split(',')
        docid = -1
        cursor = 0
        ttf = 0
        doc_freq = 0
        positions = []
        docids = []
        tf = -1
        for i in range(0, len(numbers_list)):
            if docid == -1:
                # print('Found a docid')
                docid = numbers_list[i]
            elif docid != -1 and tf == -1:
                # print('Found the TF')
                tf = numbers_list[i]
            elif docid != -1 and tf != -1:
                # print('Found the positions')
                positions.append(numbers_list[i])
                cursor += 1
                if int(cursor) == int(tf):
                    cursor = 0
                    ttf += int(tf)
                    docid = -1
                    tf = -1
                    positions = []

    # print(ttf)

    return ttf




In [53]:
get_ttf('document')

5907


5907

In [18]:
def fetch_statistics(query, inverted_file_name):
    statistics_dict = dict()
    statistics_dict['docs'] = dict()
    path = 'E:\\CS 6200\\Assignment 2\\stemmed'
    
    query_words = []
    query = query.lower()
    tokenized_words = word_tokenize(query)

    for word in tokenized_words:
        if word not in stopWords:
            query_words.append(ps.stem(word))
    with open(os.path.join(path, inverted_file_name), 'r') as f:
        for word in query_words:
            f.seek(catalog[word][0])
            # Returns line in format 'word:docid,len,positions,docid,len,positions...'
            line = f.read(catalog[word][1]) 
            # print(line)
            line = re.sub('^.*:', '', line)
            line = line.replace(';','')
            numbers_list = line.split(',')
            # print(len(numbers_list))
            docid = -1
            cursor = 0
            ttf = 0
            doc_freq = 0
            positions = []
            docids = []
            tf = -1
            for i in range(0, len(numbers_list)):
                if docid == -1:
                    # print('Found a docid')
                    docid = numbers_list[i]
                elif docid != -1 and tf == -1:
                    # print('Found the TF')
                    tf = numbers_list[i]
                elif docid != -1 and tf != -1:
                    # print('Found the positions')
                    positions.append(numbers_list[i])
                    cursor += 1
                    if int(cursor) == int(tf):
                        cursor = 0
                        ttf += int(tf)
                        doc_freq += 1
                        # print('Adding to dict...')
                        # print(docid, tf, positions)
                        doc_original_id = num_to_docid[docid]
                        if doc_original_id not in statistics_dict['docs']:
                            statistics_dict['docs'][doc_original_id] = dict()
                        if 'term_statistics' not in statistics_dict['docs'][doc_original_id]:
                            statistics_dict['docs'][doc_original_id]['term_statistics'] = dict()
                        if word not in statistics_dict['docs'][doc_original_id]['term_statistics']:
                            statistics_dict['docs'][doc_original_id]['term_statistics'][word] = dict()
                        statistics_dict['docs'][doc_original_id]['term_statistics'][word]['term_freq'] = tf
                        statistics_dict['docs'][doc_original_id]['term_statistics'][word]['positions'] = positions
                        statistics_dict['docs'][doc_original_id]['doc_length'] = get_doc_length(int(docid), doc_lengths)
                        docids.append(doc_original_id)
                        docid = -1
                        tf = -1
                        positions = []
                        # print(statistics_dict)
            for doc_original_id in docids:
                # print(doc_original_id)
                statistics_dict['docs'][doc_original_id]['term_statistics'][word]['ttf'] = ttf
                statistics_dict['docs'][doc_original_id]['term_statistics'][word]['doc_freq'] = doc_freq
            ttf = 0
            doc_freq = 0
    return statistics_dict

In [19]:
ps = PorterStemmer()

In [20]:
def tf_idf(query):
    tfidf_scores = dict()

    query_words = []
    query = query.lower()
    tokenized_words = word_tokenize(query)

    for word in tokenized_words:
        if word not in stopWords:
            query_words.append(ps.stem(word))

    statistics = fetch_statistics(query, 'l85_merged_index.txt')

    for doc in statistics['docs']:
        doc_frequency = 1
        doc_length = 0
        okapiScore = 0
        tfidf_score = 0
        tf = 0
        if statistics['docs'][doc]['term_statistics']:
            words = statistics['docs'][doc]['term_statistics']
            termsList = list(words.keys())
            doc_length = int(statistics['docs'][doc]['doc_length']) 
            for word in query_words:
                if word in termsList:
                    tf = int(words[word]["term_freq"])
                    doc_frequency = int(words[word]["doc_freq"])
                else:
                    tf = 0
                    doc_frequency = 1
                okapiScore = tf/(tf + 0.5 + ((1.5*doc_length)/avg_length))
                # idf = math.log((total_docs/doc_frequency))
                tfidf_score += okapiScore 
            tfidf_scores[doc] = tfidf_score 
    return tfidf_scores

In [21]:
def okapi_bm25(query, b, k1):
    bm25_scores = dict()

    query_words = []
    query = query.lower()
    tokenized_words = word_tokenize(query)

    for word in tokenized_words:
        if word not in stopWords:
            query_words.append(ps.stem(word))

    statistics = fetch_statistics(query, 'l85_merged_index.txt')
    
    for doc in statistics['docs']:
        doc_frequency = 0
        okapibm25_score = 0
        tf = 0
        if statistics['docs'][doc]['term_statistics']:
            words = statistics['docs'][doc]['term_statistics']
            termsList = list(words.keys())
            doc_length = int(statistics['docs'][doc]['doc_length']) 
            for word in query_words:
                if word in termsList:
                    tf = int(words[word]["term_freq"])
                    doc_frequency = int(words[word]["doc_freq"])
                else:
                    tf = 0
                    doc_frequency = 1
                okapiScore = (tf + (k1*tf))/(tf + (k1*((1-b) + (b*(doc_length/avg_length)))))
                idf = math.log((total_docs + 0.5/doc_frequency + 0.5))
                okapibm25_score += okapiScore * idf
            bm25_scores[doc] = okapibm25_score
    return bm25_scores


In [22]:
vocab_size = len(list(catalog.keys()))

In [23]:
def unigram_laplace(query, vocab_size):
    lmlaplace_scores = dict()
    query_words = []
    query = query.lower()
    tokenized_words = word_tokenize(query)

    for word in tokenized_words:
        if word not in stopWords:
            query_words.append(ps.stem(word))


    statistics = fetch_statistics(query, 'l85_merged_index.txt')
    for doc in statistics['docs']:
        tf = 0
        laplace_score = 0
        doc_length = 0
        if statistics['docs'][doc]['term_statistics']:
            words = statistics['docs'][doc]['term_statistics']
            termsList = list(words.keys())
            doc_length = int(statistics['docs'][doc]['doc_length']) 
            for word in query_words:
                if word in termsList:
                    tf = int(words[word]["term_freq"])
                else:
                    tf = 0
                p_laplace = (tf + 1)/(doc_length + vocab_size)
                laplace_score += math.log(p_laplace)

            lmlaplace_scores[doc] = laplace_score
    return lmlaplace_scores


In [92]:
ttf_dict = {}

for query in queries:
    tokenized_words = word_tokenize(queries[query].lower())

    query_words = []

    for word in tokenized_words:
        if word not in stopWords:
            query_words.append(ps.stem(word))

    for word in query_words:
        print(word)
        if word not in ttf_dict:
            ttf_dict[word] = get_ttf(word) 



document
discuss
alleg
measur
taken
corrupt
public
offici
government
jurisdict
worldwid
document
report
type
weather
event
directli
caus
least
fatal
locat
document
predict
prime
lend
rate
report
actual
prime
rate
move
document
report
incurs
land
air
water
border
area
countri
militari
forc
second
countri
guerrilla
group
base
second
countri
document
report
event
result
polit
motiv
hostag
take
document
report
militari
coup
d'etat
attempt
success
countri
document
describ
identifi
support
nation
rifl
associ
nra
asset
document
identifi
develop
iran
contra
affair
document
predict
anticip
rail
strike
report
ongo
rail
strike
document
report
poach
method
type
wildlif
document
cite
sign
contract
preliminari
agreement
make
tent
reserv
launch
commerci
satellit
document
report
current
crimin
action
offic
fail
financi
institut
document
identifi
crime
perpetr
aid
comput
document
identifi
effort
non
communist
industri
state
regul
transfer
high
tech
good
technolog
undesir
nation
document
identifi
exist


In [97]:
ttf_dict['mci']

267

In [106]:
def unigram_jm(query, vocab_size, lam):

    jm_scores = dict()
    query_words = []
    query = query.lower()
    tokenized_words = word_tokenize(query)

    for word in tokenized_words:
        if word not in stopWords:
            query_words.append(ps.stem(word))


    statistics = fetch_statistics(query, 'l85_merged_index.txt')
    for doc in statistics['docs']:
        # print('DOC:', doc)
        tf = 0
        lmjm_score = 0
        doc_length = 0
        if statistics['docs'][doc]['term_statistics']:
            words = statistics['docs'][doc]['term_statistics']
            termsList = list(words.keys())
            doc_length = int(statistics['docs'][doc]['doc_length']) 
            for word in query_words:
                ttf = ttf_dict[word]
                # print('Size, word: ',len(query_words), word)
                if word in termsList:
                    tf = int(words[word]["term_freq"])
                    p_jm = (lam*(tf/doc_length)) + ((1-lam)* ttf/vocab_size)
                    # print('PJM: ', p_jm)
                    lmjm_score += math.log(p_jm)
                else:
                    # print(word)
                    lmjm_score += math.log(((1-lam)* ttf/vocab_size))

            jm_scores[doc] = lmjm_score
    print('Exiting JM function')
    return jm_scores
    # jm_scores = dict()
    # query_words = []
    # query = query.lower()
    # tokenized_words = word_tokenize(query)

    # for word in tokenized_words:
    #     if word not in stopWords:
    #         query_words.append(ps.stem(word))

    # print('Initializing...')
    # # print('Query words: ', query_words)


    # statistics = fetch_statistics(query, 'l85_merged_index.txt')
    # for doc in statistics['docs']:
    #     tf = 0
    #     lmjm_score = 0
    #     doc_length = 0
    #     if statistics['docs'][doc]['term_statistics']:
    #         words = statistics['docs'][doc]['term_statistics']
    #         termsList = list(words.keys())
    #         doc_length = int(statistics['docs'][doc]['doc_length'])
    #         print('Query: ', query_words)

    #         for word in query_words:
    #             print('Query word ', word)
    #             if word in termsList:
    #                 tf = int(words[word]["term_freq"])
    #                 p_jm = (lam*(tf/doc_length)) + ((1-lam)* tf/vocab_size)
    #                 lmjm_score += math.log(p_jm)
    #             else:
    #                 lmjm_score += math.log(((1-lam)* tf/vocab_size))

    #         jm_scores[doc] = lmjm_score
    # return jm_scores



In [25]:
def write_to_file( file, query_id, doclist, dict):
    outF = open(file, "a")
    string = ""
    for i in range(0, len(doclist)):
        string = str(query_id) + " Q0 " + doclist[i] + "  " + str(i+1) + "  "+str(dict[doclist[i]]) + " Exp\n" 
        outF.write(string)
    outF.close()

In [111]:
def run_query_and_write_results(query, query_id, filename, choice):
    score_dict = dict()
    if choice == 1:
        score_dict = tf_idf(query)
    elif choice == 2:
        score_dict = okapi_bm25(query, 0.75, 1.2)
    elif choice == 3:
        score_dict = unigram_laplace(query, vocab_size)
    elif choice == 4:
        score_dict = unigram_jm(query, vocab_size, 0.95)

    sorted_d = dict( sorted(score_dict.items(), key=operator.itemgetter(1),reverse=True))
    sortedKeys = list(sorted_d)

    # Write scores to File
    write_to_file(filename, query_id, sortedKeys, sorted_d)

In [27]:
query_ids = list(queries.keys())

# TF-IDF

In [28]:
# for i in range(0, len(query_ids)):
#     run_query_and_write_results(queries[query_ids[i]], query_ids[i], 'E:\\CS 6200\\Assignment 2\\results\\a6_tfidf_stemmed.txt', 1)

# BM 25

In [29]:
# for i in range(0, len(query_ids)):
#     run_query_and_write_results(queries[query_ids[i]], query_ids[i], 'E:\\CS 6200\\Assignment 2\\results\\a6_bm25_stemmed.txt', 2)

# Unigram Laplace

In [30]:
# for i in range(0, len(query_ids)):
#     run_query_and_write_results(queries[query_ids[i]], query_ids[i], 'E:\\CS 6200\\Assignment 2\\results\\a6_laplace_stemmed.txt', 3)

# Unigram JM

In [112]:
for i in range(0, len(query_ids)):
    print('Query ', i)
    run_query_and_write_results(queries[query_ids[i]], query_ids[i], 'E:\\CS 6200\\Assignment 2\\results\\a6_jm5_stemmed.txt', 4)

Query  0
Exiting JM function
Query  1
Exiting JM function
Query  2
Exiting JM function
Query  3
Exiting JM function
Query  4
Exiting JM function
Query  5
Exiting JM function
Query  6
Exiting JM function
Query  7
Exiting JM function
Query  8
Exiting JM function
Query  9
Exiting JM function
Query  10
Exiting JM function
Query  11
Exiting JM function
Query  12
Exiting JM function
Query  13
Exiting JM function
Query  14
Exiting JM function
Query  15
Exiting JM function
Query  16
Exiting JM function
Query  17
Exiting JM function
Query  18
Exiting JM function
Query  19
Exiting JM function
Query  20
Exiting JM function
Query  21
Exiting JM function
Query  22
Exiting JM function
Query  23
Exiting JM function
Query  24
Exiting JM function
